In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from sklearn.model_selection import train_test_split
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import cv2
from random import shuffle
import random
import glob
from skimage.transform import resize
#from PIL import Image
import os
from model import *
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from collections import defaultdict
import itertools
from tqdm import tqdm
import time
import shutil


In [ ]:
#from inception_resnet_v2 import *

In [ ]:
num_classes = 11

In [ ]:
def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size
            if (batch_end>len(data_y)):
                batch_end = len(data_y)

In [ ]:
def read_landUse(dataset_path, im_type):
    import glob
    paths_X = []   
    labels = [] 
    i = 0
    folders = sorted(os.listdir(dataset_path))
    for folder in folders:
        temp = sorted (glob.glob(os.path.join(dataset_path,folder+'/*'+im_type)))
        for k in range(len(temp)):
            labels.append (i)
        paths_X += temp
        i += 1
        
    if(len(paths_X)==0):
        print ('Dataset could not found. Please provide correct path.')
    return paths_X, labels

In [ ]:
dataset_path = 'train'

paths, labels = read_landUse(dataset_path, '[(.png)(.jpg)]')

In [ ]:
#plt.imshow(np.array(cv2.imread(paths[0])))
images = np.array([cv2.imread(image_name) for image_name in paths]) 
#images[0]
#np.shape(images)

In [ ]:
model_name = 'InceptionResNet-v2'
#dataset_path = '/media/waseem/3D0A87CA7D9EC477/M.Bhimra/mohbat_resnet/FINAL DATASET/dataset/houses'
#model_name = 'resNet152_retrain_less'
#dataset_path = 'train-64'

In [ ]:
X = images
y = to_categorical(labels, num_classes)
#X, y = load_dataset(dataset_path)
print ('Size of dataset:', len(y))

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Display dataset stats

In [ ]:
folders = os.listdir(dataset_path)
names, counts = [], []
for i, folder in enumerate(folders):
    dir_path = os.path.join(dataset_path, folder)
    images1 = os.listdir(dir_path)
    names.append(folder)
    counts.append(len(images1))
    print(i, folder, len(images1))
df = pd.DataFrame(data={'Name': names, 'Count': counts})
ax = df.plot(kind='bar', xticks=counts, grid=True, legend=False, figsize=(16,6))
ax.set_xlabel('Classes', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
plt.xticks(np.arange(len(names)), names, rotation=45, fontsize=15)
for i, v in enumerate(counts):
    ax.text(i-.18, v+15, str(v), color='blue', fontweight='bold')
plt.show()

In [ ]:
input_shape=(256,256,3)
# The '-1' is because we add a new class for finetuning
#model = resNet(input_shape, num_classes, model_type='resnet_152')
#model = VGG16(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
model =InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)

#model = resNet(input_shape, num_classes, model_type='resnet_34')
#model.load_weights(model_name+'.h5')
# model.load_weights('resNet152 (another copy).h5')

# # FINE TUNING HERE
# top_model = Sequential()
# top_model.add(Dense(input_shape=model.layers[-2].output_shape, units=num_classes, rnel
# kernel_initializer="he_normal", activation="softmax"))

# model.layers.pop()
# model.outputs = [model.layers[-1].output]
# model.layers[-1].outbound_nodes = []

# model = Model(inputs=model.inputs, outputs=top_model(model.outputs[0]))
# # for layer in model.layers[:-1]:
# #     layer.trainable = False

model.summary()
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# plot_model(model, to_file='model.png')


In [ ]:
class_weights = {0: 648/618,
                1: 648/600,
                2: 648/599,
                3: 648/599,
                4: 648/594,
                5: 648/603,
                6: 648/648,
                7: 648/593,
                8: 648/582,
                9: 648/600,
               10: 648/742
               }

In [ ]:
print(class_weights)

In [ ]:
batch_size = 30

Train ResNet-152 network using "fit_generator"

In [ ]:
checkpoint = ModelCheckpoint('WEIGHTS/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
tensorboard = TensorBoard(log_dir='TENSORBOARD/', batch_size=batch_size, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.00001)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=2, mode='auto')
cvslogger = keras.callbacks.CSVLogger('WEIGHTS/'+model_name+'.csv', separator=',', append=True)
nb_epoch = 200
callbacks = [checkpoint, reduce_lr, earlystop, cvslogger, tensorboard]
#train_steps = int(len(y_train)//batch_size)
#val_steps = int(len(y_test)//batch_size)
kf = KFold(n_splits=10)
kf.get_n_splits(X)

#print(kf)  
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    train_gen = batch_generator(X_train, y_train, batch_size)
    test_gen = batch_generator(X_test, y_test, batch_size)
    history = model.fit_generator(train_gen, int(len(y_train)//batch_size), epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=test_gen, 
                    validation_steps=int(len(y_test)//batch_size), shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks, class_weight=class_weights)
    #model.evaluate_generator(test_gen, steps=len(X[test])//batch_size


# sanity check

In [ ]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

In [ ]:
def tuned_model(model, x):

    t_model = Model(model.input, x)
    
    return t_model


layer_dict = dict([(layer.name, layer) for layer in model.layers])
x = layer_dict['avg_pool'].output 

t_model = tuned_model(model, x)

In [ ]:
p_output = t_model.predict(X)
val_labels= np.argmax(y, axis=1)

import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot as plt
from tsne import bh_sne

X = np.asarray(p_output).astype('float64')
X = X.reshape((p_output.shape[0], -1))
# For speed of computation, only run on a subset
#n = 20000
#X_train_data = X_train[:n]
#Y_train_data = y_train[:n]
# perform t-SNE embedding
vis_data = bh_sne(X)
# plot the result
vis_x = vis_data[:, 0]
vis_y = vis_data[:, 1]
f = plt.figure()
plt.scatter(vis_x, vis_y, c=val_labels, cmap=plt.cm.get_cmap("jet", 11), s=10)
plt.colorbar(ticks=range(11))
plt.show()
f.savefig("tsne_BrickKilns.pdf")

In [ ]:
Tdataset_path = 'test'
Tpaths, Tlabels = read_landUse(Tdataset_path, '[(.png)(.jpg)]')
Timages = np.array([cv2.imread(image_name) for image_name in Tpaths]) 
X_test = Timages
y_test = to_categorical(Tlabels, num_classes)

In [ ]:
y_pred = model.predict(X_test)

class_labels = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground',5: 'houses',6:'kiln',7:'orchard',8:'parking',9:'roads',10:'unevenland'}

In [ ]:
preds=y_pred
print(preds.shape)

In [ ]:
pres_classes = []
for p in preds.argmax(axis=1):
    pres_classes.append(class_labels[p])
#print(pres_classes)

In [ ]:
truth_classes = []
for p in y_test.argmax(axis=1):
    truth_classes.append(class_labels[p])
#print(truth_classes)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def my_confusion_matrix(y_test, y_pred, classes):
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=classes,title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=classes, normalize=True,title='Normalized confusion matrix')

    plt.show()
    return cnf_matrix

In [ ]:
cm = my_confusion_matrix(truth_classes,pres_classes,class_labels)

In [ ]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()
# Precision Calculation https://www.python-course.eu/confusion_matrix.php
# Precision Calculation https://www.python-course.eu/confusion_matrix.php
a = precision(6,cm)
b = recall(6,cm)
print('Precision', a)
print('Recall', b)
print('F1', (2*a*b)/(a+b))

In [ ]:
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [ ]:
#model = load_model("InceptionResNet-v2.h5")
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

In [ ]:
np.shape(X)

In [ ]:
activations = activation_model.predict(X[1].reshape(1,24,64,1))
display_activation(activations, 3, 2, 3)

In [ ]:
l1 = model.layers[1]
plt.imshow(l1.get_weights()[0][:,:,0,0].squeeze(), cmap='gray')

In [ ]:
l2 = model.layers[4]
plt.imshow(l2.get_weights()[0][:,:,0,0].squeeze(), cmap='gray')

In [ ]:
top_layer.shape

In [ ]:
layer_outputs = [layer.output for layer in model.layers[:12]] # Extracts the outputs of the top 12 layers
activation_model = Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input
activations = activation_model.predict([test_data_1[0].reshape((1,22,22,1)),test_data_2[0].reshape((1,22,22,1))]) # Returns a list of five Numpy arrays: one array per layer activation

In [ ]:
first_layer_activation = activations[3]
print(first_layer_activation.shape)
plt.matshow(first_layer_activation[0, :, :, 0], cmap='viridis')

In [ ]:
####################################################
# test_images_dir = 'lhr/newtask'
test_images_dir = 'Kabul2NepalDatasets/Kabul2NepalZoom20Y2018_1_1000_Images/'

# results_dir = 'results_lahore_17_with_old_wets_attempt2'
results_dir = 'Results/Kabul2NepalZoom20Y2018_1_1000_Images/'
if not os.path.exists(results_dir):
    print('Creating new results directory "{}"'.format(results_dir))
    os.mkdir(results_dir)
im_dir = os.path.join(results_dir, 'images')
if not os.path.exists(im_dir):
    os.mkdir(im_dir)

#for file in os.listdir(test_images_dir):
#    curr_name, curr_ext = os.path.splitext(file)
 #   print(curr_name[:-1])
 #   break
#    new_name = curr_name[:-1]
#    os.rename(os.path.join(test_images_dir, file), os.path.join(test_images_dir, new_name+curr_ext))

########################################################
########### RENAMING DONE IN DOWNLOAD SCRIPT ###########
########################################################

Testing the model and creating result files

In [ ]:
## print('Waiting'len(images)len(images)len(images), end='', flush=True)len(images)
# while True:
    
 #   if len(os.listdir(test_images_dir)) != 544244:
 #      time.sleep(10*60)
 #       print('.', end='', flush=True)
 #       continue


# if os.path.exists(os.path.join(results_dir, 'houses.txt')) or os.path.exists(os.path.join(results_dir, 'test.csv')):
#     raise OSError('Result files already present, this script will append to the existing data.')
    
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms', 14: 'orchard'}
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms'}
# label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'mosque', 8: 'oiltanks', 9: 'orchard', 10: 'parking', 11: 'parks', 12: 'ponds', 13: 'roads', 14: 'tennis'}
label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'orchard', 8: 'parking', 9: 'roads', 10:'unevenland'}
y_tiles, x_tiles = [], []
label_probs = defaultdict(list)
top_label_probs = defaultdict(list)

# # Resuming check
# try:
#     with open(os.path.join(results_dir, 'last_file_done.txt'), 'r') as f:
#         loop_index = int(f.read())
#     print('Resuming from file index "{}"'.format(loop_index))
# except FileNotFoundError:
#     loop_index = 0
#     print('Starting fresh from file index "0"')

filenames = os.listdir(test_images_dir)

# # Missing images check
# try:
#     with open(os.path.join(results_dir, 'missing_images.txt'), 'r') as f:
#         missing_images = [name.strip() for name in f.readlines()]
#     print('Continuing with {} missing images.'.format(len(missing_images)))
# except FileNotFoundError:
#     missing_images = []
#     print('Starting fresh with empty missing images list')
for img_name in tqdm(filenames):


#     extt=img_name.split('.')[1]
    orig_name, extt = os.path.splitext(img_name)
    if(extt == '.jpg'):
        img_orig = cv2.imread(os.path.join(test_images_dir, img_name), cv2.IMREAD_COLOR)
        # To catch corrupt images
        if type(img_orig) == type(None):
            print('Skipping an image "{}"'.format(img_name))
        #    missing_images.append(img_name)
            continue

        img = np.expand_dims(img_orig, axis=0)
        class_name = img_name.split('.')[0]

        pred = model.predict(img, verbose=0)
    #    preds.append((pred.argmax(), pred.max(), class_name))

#         src = os.path.join(test_images_dir, img_name)
#         dst = os.path.join(results_dir, 'images', orig_name+'_'+label_names[pred.argmax()]+extt)
#         shutil.copyfile(src, dst)
#         print(img_name, dst)

#         break
        x_tile, y_tile = os.path.splitext(img_name)[0].split('_')
        y_tiles.append(y_tile)
        x_tiles.append(x_tile)

        # Append to individual results files on each itteration
        for i, label in label_names.items():
            file_path = os.path.join(results_dir, label+'.txt')
            label_probs[label].append(pred[0, i])
            with open(file_path, 'a') as f:
                f.write('{} {} {}\n'.format(y_tile, x_tile, pred[0, i]))
#         # Append to full result file on every 1000 itterations
#         #if loop_index % 1000 == 0:
#         d = {'y_tile': y_tiles, 'x_tile': x_tiles}
#         d.update(label_probs)
#         final_df = pd.DataFrame(data=d)
#         final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
        # Resuming 
        #with open('last_file_done.txt', 'w') as f:
         #   f.write(str(loop_index))
        # Write names of corrupted files for later use
        #with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
           # for i in missing_images:
           #     f.write('{}\n'.format(i))
        #loop_index += 1

        top3_inds = np.argsort(pred)
        top3_inds = np.flip(top3_inds, axis=1)
        top3_labels = [ label_names[i] for ia in top3_inds for i in ia ]
        top3_probs = pred[0, top3_inds]
#     print(top3_labels)

        for n in range(num_classes):
            top_label_probs['label'+str(n+1)].append(top3_labels[n])
            top_label_probs['prob'+str(n+1)].append(top3_probs[0, n])

# Append to full result file last time
d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(label_probs)
final_df = pd.DataFrame(data=d)
final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
# final_df

# Write names of corrupted files for later use
#with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
#    for i in missing_images:
#        f.write('{}\n'.format(i))

d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(top_label_probs)
sorted_df = pd.DataFrame(data=d)
sorted_df = sorted_df[['y_tile', 'x_tile', 'label1', 'prob1', 'label2', 'prob2', 'label3', 'prob3', 'label4', 'prob4', 'label5', 'prob5', 'label6', 'prob6', 'label7', 'prob7', 'label8', 'prob8', 'label9', 'prob9', 'label10', 'prob10']]
sorted_df.to_csv(os.path.join(results_dir, 'top_predictions.csv'), index=False, header=True)
# sorted_df.head()

# preds = sorted(preds, key=lambda x: x[0])
# for p_class, p_prob, truth in preds:
#     got_it = label_names[p_class] == truth
#     print('{}\tPrediction: {}\tTruth: {}\tProb: {}'.format(got_it, label_names[p_class], truth, p_prob))
#     break

In [ ]:
####################################################
# test_images_dir = 'lhr/newtask'
test_images_dir = 'Kabul2NepalDatasets/Kabul2NepalZoom20Y2018_1001_5000_Images/'

# results_dir = 'results_lahore_17_with_old_wets_attempt2'
results_dir = 'Results/Kabul2NepalZoom20Y2018_1001_5000_Images/'
if not os.path.exists(results_dir):
    print('Creating new results directory "{}"'.format(results_dir))
    os.mkdir(results_dir)
im_dir = os.path.join(results_dir, 'images')
if not os.path.exists(im_dir):
    os.mkdir(im_dir)

#for file in os.listdir(test_images_dir):
#    curr_name, curr_ext = os.path.splitext(file)
 #   print(curr_name[:-1])
 #   break
#    new_name = curr_name[:-1]
#    os.rename(os.path.join(test_images_dir, file), os.path.join(test_images_dir, new_name+curr_ext))

########################################################
########### RENAMING DONE IN DOWNLOAD SCRIPT ###########
########################################################

In [ ]:
## print('Waiting'len(images)len(images)len(images), end='', flush=True)len(images)
# while True:
    
 #   if len(os.listdir(test_images_dir)) != 544244:
 #      time.sleep(10*60)
 #       print('.', end='', flush=True)
 #       continue


# if os.path.exists(os.path.join(results_dir, 'houses.txt')) or os.path.exists(os.path.join(results_dir, 'test.csv')):
#     raise OSError('Result files already present, this script will append to the existing data.')
    
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms', 14: 'orchard'}
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms'}
# label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'mosque', 8: 'oiltanks', 9: 'orchard', 10: 'parking', 11: 'parks', 12: 'ponds', 13: 'roads', 14: 'tennis'}
label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'orchard', 8: 'parking', 9: 'roads', 10:'unevenland'}
y_tiles, x_tiles = [], []
label_probs = defaultdict(list)
top_label_probs = defaultdict(list)

# # Resuming check
# try:
#     with open(os.path.join(results_dir, 'last_file_done.txt'), 'r') as f:
#         loop_index = int(f.read())
#     print('Resuming from file index "{}"'.format(loop_index))
# except FileNotFoundError:
#     loop_index = 0
#     print('Starting fresh from file index "0"')

filenames = os.listdir(test_images_dir)

# # Missing images check
# try:
#     with open(os.path.join(results_dir, 'missing_images.txt'), 'r') as f:
#         missing_images = [name.strip() for name in f.readlines()]
#     print('Continuing with {} missing images.'.format(len(missing_images)))
# except FileNotFoundError:
#     missing_images = []
#     print('Starting fresh with empty missing images list')
for img_name in tqdm(filenames):


#     extt=img_name.split('.')[1]
    orig_name, extt = os.path.splitext(img_name)
    if(extt == '.jpg'):
        img_orig = cv2.imread(os.path.join(test_images_dir, img_name), cv2.IMREAD_COLOR)
        # To catch corrupt images
        if type(img_orig) == type(None):
            print('Skipping an image "{}"'.format(img_name))
        #    missing_images.append(img_name)
            continue

        img = np.expand_dims(img_orig, axis=0)
        class_name = img_name.split('.')[0]

        pred = model.predict(img, verbose=0)
    #    preds.append((pred.argmax(), pred.max(), class_name))

#         src = os.path.join(test_images_dir, img_name)
#         dst = os.path.join(results_dir, 'images', orig_name+'_'+label_names[pred.argmax()]+extt)
#         shutil.copyfile(src, dst)
#         print(img_name, dst)

#         break
        x_tile, y_tile = os.path.splitext(img_name)[0].split('_')
        y_tiles.append(y_tile)
        x_tiles.append(x_tile)

        # Append to individual results files on each itteration
        for i, label in label_names.items():
            file_path = os.path.join(results_dir, label+'.txt')
            label_probs[label].append(pred[0, i])
            with open(file_path, 'a') as f:
                f.write('{} {} {}\n'.format(y_tile, x_tile, pred[0, i]))
#         # Append to full result file on every 1000 itterations
#         #if loop_index % 1000 == 0:
#         d = {'y_tile': y_tiles, 'x_tile': x_tiles}
#         d.update(label_probs)
#         final_df = pd.DataFrame(data=d)
#         final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
        # Resuming 
        #with open('last_file_done.txt', 'w') as f:
         #   f.write(str(loop_index))
        # Write names of corrupted files for later use
        #with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
           # for i in missing_images:
           #     f.write('{}\n'.format(i))
        #loop_index += 1

        top3_inds = np.argsort(pred)
        top3_inds = np.flip(top3_inds, axis=1)
        top3_labels = [ label_names[i] for ia in top3_inds for i in ia ]
        top3_probs = pred[0, top3_inds]
#     print(top3_labels)

        for n in range(num_classes):
            top_label_probs['label'+str(n+1)].append(top3_labels[n])
            top_label_probs['prob'+str(n+1)].append(top3_probs[0, n])

# Append to full result file last time
d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(label_probs)
final_df = pd.DataFrame(data=d)
final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
# final_df

# Write names of corrupted files for later use
#with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
#    for i in missing_images:
#        f.write('{}\n'.format(i))

d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(top_label_probs)
sorted_df = pd.DataFrame(data=d)
sorted_df = sorted_df[['y_tile', 'x_tile', 'label1', 'prob1', 'label2', 'prob2', 'label3', 'prob3', 'label4', 'prob4', 'label5', 'prob5', 'label6', 'prob6', 'label7', 'prob7', 'label8', 'prob8', 'label9', 'prob9', 'label10', 'prob10']]
sorted_df.to_csv(os.path.join(results_dir, 'top_predictions.csv'), index=False, header=True)
# sorted_df.head()

# preds = sorted(preds, key=lambda x: x[0])
# for p_class, p_prob, truth in preds:
#     got_it = label_names[p_class] == truth
#     print('{}\tPrediction: {}\tTruth: {}\tProb: {}'.format(got_it, label_names[p_class], truth, p_prob))
#     break

In [ ]:
####################################################
# test_images_dir = 'lhr/newtask'
test_images_dir = 'Kabul2NepalDatasets/Kabul2NepalZoom20Y2018_5001_10000_Images/'

# results_dir = 'results_lahore_17_with_old_wets_attempt2'
results_dir = 'Results/Kabul2NepalZoom20Y2018_5001_10000_Images/'
if not os.path.exists(results_dir):
    print('Creating new results directory "{}"'.format(results_dir))
    os.mkdir(results_dir)
im_dir = os.path.join(results_dir, 'images')
if not os.path.exists(im_dir):
    os.mkdir(im_dir)

#for file in os.listdir(test_images_dir):
#    curr_name, curr_ext = os.path.splitext(file)
 #   print(curr_name[:-1])
 #   break
#    new_name = curr_name[:-1]
#    os.rename(os.path.join(test_images_dir, file), os.path.join(test_images_dir, new_name+curr_ext))

########################################################
########### RENAMING DONE IN DOWNLOAD SCRIPT ###########
########################################################

In [ ]:
## print('Waiting'len(images)len(images)len(images), end='', flush=True)len(images)
# while True:
    
 #   if len(os.listdir(test_images_dir)) != 544244:
 #      time.sleep(10*60)
 #       print('.', end='', flush=True)
 #       continue


# if os.path.exists(os.path.join(results_dir, 'houses.txt')) or os.path.exists(os.path.join(results_dir, 'test.csv')):
#     raise OSError('Result files already present, this script will append to the existing data.')
    
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms', 14: 'orchard'}
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms'}
# label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'mosque', 8: 'oiltanks', 9: 'orchard', 10: 'parking', 11: 'parks', 12: 'ponds', 13: 'roads', 14: 'tennis'}
label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'orchard', 8: 'parking', 9: 'roads', 10:'unevenland'}
y_tiles, x_tiles = [], []
label_probs = defaultdict(list)
top_label_probs = defaultdict(list)

# # Resuming check
# try:
#     with open(os.path.join(results_dir, 'last_file_done.txt'), 'r') as f:
#         loop_index = int(f.read())
#     print('Resuming from file index "{}"'.format(loop_index))
# except FileNotFoundError:
#     loop_index = 0
#     print('Starting fresh from file index "0"')

filenames = os.listdir(test_images_dir)

# # Missing images check
# try:
#     with open(os.path.join(results_dir, 'missing_images.txt'), 'r') as f:
#         missing_images = [name.strip() for name in f.readlines()]
#     print('Continuing with {} missing images.'.format(len(missing_images)))
# except FileNotFoundError:
#     missing_images = []
#     print('Starting fresh with empty missing images list')
for img_name in tqdm(filenames):


#     extt=img_name.split('.')[1]
    orig_name, extt = os.path.splitext(img_name)
    if(extt == '.jpg'):
        img_orig = cv2.imread(os.path.join(test_images_dir, img_name), cv2.IMREAD_COLOR)
        # To catch corrupt images
        if type(img_orig) == type(None):
            print('Skipping an image "{}"'.format(img_name))
        #    missing_images.append(img_name)
            continue

        img = np.expand_dims(img_orig, axis=0)
        class_name = img_name.split('.')[0]

        pred = model.predict(img, verbose=0)
    #    preds.append((pred.argmax(), pred.max(), class_name))

#         src = os.path.join(test_images_dir, img_name)
#         dst = os.path.join(results_dir, 'images', orig_name+'_'+label_names[pred.argmax()]+extt)
#         shutil.copyfile(src, dst)
#         print(img_name, dst)

#         break
        x_tile, y_tile = os.path.splitext(img_name)[0].split('_')
        y_tiles.append(y_tile)
        x_tiles.append(x_tile)

        # Append to individual results files on each itteration
        for i, label in label_names.items():
            file_path = os.path.join(results_dir, label+'.txt')
            label_probs[label].append(pred[0, i])
            with open(file_path, 'a') as f:
                f.write('{} {} {}\n'.format(y_tile, x_tile, pred[0, i]))
#         # Append to full result file on every 1000 itterations
#         #if loop_index % 1000 == 0:
#         d = {'y_tile': y_tiles, 'x_tile': x_tiles}
#         d.update(label_probs)
#         final_df = pd.DataFrame(data=d)
#         final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
        # Resuming 
        #with open('last_file_done.txt', 'w') as f:
         #   f.write(str(loop_index))
        # Write names of corrupted files for later use
        #with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
           # for i in missing_images:
           #     f.write('{}\n'.format(i))
        #loop_index += 1

        top3_inds = np.argsort(pred)
        top3_inds = np.flip(top3_inds, axis=1)
        top3_labels = [ label_names[i] for ia in top3_inds for i in ia ]
        top3_probs = pred[0, top3_inds]
#     print(top3_labels)

        for n in range(num_classes):
            top_label_probs['label'+str(n+1)].append(top3_labels[n])
            top_label_probs['prob'+str(n+1)].append(top3_probs[0, n])

# Append to full result file last time
d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(label_probs)
final_df = pd.DataFrame(data=d)
final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
# final_df

# Write names of corrupted files for later use
#with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
#    for i in missing_images:
#        f.write('{}\n'.format(i))

d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(top_label_probs)
sorted_df = pd.DataFrame(data=d)
sorted_df = sorted_df[['y_tile', 'x_tile', 'label1', 'prob1', 'label2', 'prob2', 'label3', 'prob3', 'label4', 'prob4', 'label5', 'prob5', 'label6', 'prob6', 'label7', 'prob7', 'label8', 'prob8', 'label9', 'prob9', 'label10', 'prob10']]
sorted_df.to_csv(os.path.join(results_dir, 'top_predictions.csv'), index=False, header=True)
# sorted_df.head()

# preds = sorted(preds, key=lambda x: x[0])
# for p_class, p_prob, truth in preds:
#     got_it = label_names[p_class] == truth
#     print('{}\tPrediction: {}\tTruth: {}\tProb: {}'.format(got_it, label_names[p_class], truth, p_prob))
#     break

In [ ]:
####################################################
# test_images_dir = 'lhr/newtask'
test_images_dir = 'Kabul2NepalDatasets/Kabul2NepalZoom20Y2018_10000_15000_Images/'

# results_dir = 'results_lahore_17_with_old_wets_attempt2'
results_dir = 'Kabul2NepalDatasets/Kabul2NepalZoom20Y2018_10000_15000_Images/'
if not os.path.exists(results_dir):
    print('Creating new results directory "{}"'.format(results_dir))
    os.mkdir(results_dir)
im_dir = os.path.join(results_dir, 'images')
if not os.path.exists(im_dir):
    os.mkdir(im_dir)

#for file in os.listdir(test_images_dir):
#    curr_name, curr_ext = os.path.splitext(file)
 #   print(curr_name[:-1])
 #   break
#    new_name = curr_name[:-1]
#    os.rename(os.path.join(test_images_dir, file), os.path.join(test_images_dir, new_name+curr_ext))

########################################################
########### RENAMING DONE IN DOWNLOAD SCRIPT ###########
########################################################

In [ ]:
## print('Waiting'len(images)len(images)len(images), end='', flush=True)len(images)
# while True:
    
 #   if len(os.listdir(test_images_dir)) != 544244:
 #      time.sleep(10*60)
 #       print('.', end='', flush=True)
 #       continue


# if os.path.exists(os.path.join(results_dir, 'houses.txt')) or os.path.exists(os.path.join(results_dir, 'test.csv')):
#     raise OSError('Result files already present, this script will append to the existing data.')
    
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms', 14: 'orchard'}
# label_names = {0: 'parking', 1: 'parks', 2: 'ground', 3: 'houses', 4: 'roads', 5: 'mosque', 6: 'densetrees', 7: 'kiln', 8: 'oiltanks', 9: 'tennis', 10: 'ponds', 11: 'grass', 12: 'blackfarms', 13: 'farms'}
# label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'mosque', 8: 'oiltanks', 9: 'orchard', 10: 'parking', 11: 'parks', 12: 'ponds', 13: 'roads', 14: 'tennis'}
label_names = {0: 'blackfarms', 1: 'densetrees', 2: 'farms', 3: 'grass', 4: 'ground', 5: 'houses', 6: 'kiln', 7: 'orchard', 8: 'parking', 9: 'roads', 10:'unevenland'}
y_tiles, x_tiles = [], []
label_probs = defaultdict(list)
top_label_probs = defaultdict(list)

# # Resuming check
# try:
#     with open(os.path.join(results_dir, 'last_file_done.txt'), 'r') as f:
#         loop_index = int(f.read())
#     print('Resuming from file index "{}"'.format(loop_index))
# except FileNotFoundError:
#     loop_index = 0
#     print('Starting fresh from file index "0"')

filenames = os.listdir(test_images_dir)

# # Missing images check
# try:
#     with open(os.path.join(results_dir, 'missing_images.txt'), 'r') as f:
#         missing_images = [name.strip() for name in f.readlines()]
#     print('Continuing with {} missing images.'.format(len(missing_images)))
# except FileNotFoundError:
#     missing_images = []
#     print('Starting fresh with empty missing images list')
for img_name in tqdm(filenames):


#     extt=img_name.split('.')[1]
    orig_name, extt = os.path.splitext(img_name)
    if(extt == '.jpg'):
        img_orig = cv2.imread(os.path.join(test_images_dir, img_name), cv2.IMREAD_COLOR)
        # To catch corrupt images
        if type(img_orig) == type(None):
            print('Skipping an image "{}"'.format(img_name))
        #    missing_images.append(img_name)
            continue

        img = np.expand_dims(img_orig, axis=0)
        class_name = img_name.split('.')[0]

        pred = model.predict(img, verbose=0)
    #    preds.append((pred.argmax(), pred.max(), class_name))

#         src = os.path.join(test_images_dir, img_name)
#         dst = os.path.join(results_dir, 'images', orig_name+'_'+label_names[pred.argmax()]+extt)
#         shutil.copyfile(src, dst)
#         print(img_name, dst)

#         break
        x_tile, y_tile = os.path.splitext(img_name)[0].split('_')
        y_tiles.append(y_tile)
        x_tiles.append(x_tile)

        # Append to individual results files on each itteration
        for i, label in label_names.items():
            file_path = os.path.join(results_dir, label+'.txt')
            label_probs[label].append(pred[0, i])
            with open(file_path, 'a') as f:
                f.write('{} {} {}\n'.format(y_tile, x_tile, pred[0, i]))
#         # Append to full result file on every 1000 itterations
#         #if loop_index % 1000 == 0:
#         d = {'y_tile': y_tiles, 'x_tile': x_tiles}
#         d.update(label_probs)
#         final_df = pd.DataFrame(data=d)
#         final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
        # Resuming 
        #with open('last_file_done.txt', 'w') as f:
         #   f.write(str(loop_index))
        # Write names of corrupted files for later use
        #with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
           # for i in missing_images:
           #     f.write('{}\n'.format(i))
        #loop_index += 1

        top3_inds = np.argsort(pred)
        top3_inds = np.flip(top3_inds, axis=1)
        top3_labels = [ label_names[i] for ia in top3_inds for i in ia ]
        top3_probs = pred[0, top3_inds]
#     print(top3_labels)

        for n in range(num_classes):
            top_label_probs['label'+str(n+1)].append(top3_labels[n])
            top_label_probs['prob'+str(n+1)].append(top3_probs[0, n])

# Append to full result file last time
d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(label_probs)
final_df = pd.DataFrame(data=d)
final_df.to_csv(os.path.join(results_dir, 'test.csv'), index=False, header=True)
# final_df

# Write names of corrupted files for later use
#with open(os.path.join(results_dir, 'missing.txt'), 'w') as f:
#    for i in missing_images:
#        f.write('{}\n'.format(i))

d = {'y_tile': y_tiles, 'x_tile': x_tiles}
d.update(top_label_probs)
sorted_df = pd.DataFrame(data=d)
sorted_df = sorted_df[['y_tile', 'x_tile', 'label1', 'prob1', 'label2', 'prob2', 'label3', 'prob3', 'label4', 'prob4', 'label5', 'prob5', 'label6', 'prob6', 'label7', 'prob7', 'label8', 'prob8', 'label9', 'prob9', 'label10', 'prob10']]
sorted_df.to_csv(os.path.join(results_dir, 'top_predictions.csv'), index=False, header=True)
# sorted_df.head()

# preds = sorted(preds, key=lambda x: x[0])
# for p_class, p_prob, truth in preds:
#     got_it = label_names[p_class] == truth
#     print('{}\tPrediction: {}\tTruth: {}\tProb: {}'.format(got_it, label_names[p_class], truth, p_prob))
#     break